# 2d image manifold results

## Setup

In [1]:
import sys
import numpy as np
from itertools import product


## Load results

In [2]:
n_runs = 10

In [3]:
algo_filenames = []
algo_additionals = []
algo_labels = []
algo_dividers = []

def add_algo(filename, add, label):
    algo_filenames.append(filename)
    algo_additionals.append(add)
    algo_labels.append(label)
    
    
def add_divider():
    algo_dividers.append(len(algo_filenames))

add_algo("flow", "april", r"\af{}")
add_algo("pie", "april", r"\pie{}")
add_algo("mf", "april", r"\mfmfmd{}")
add_algo("emf", "april", r"\mfmfemd{}")

n_algos = len(algo_filenames)


In [4]:
def load(name, shape, numpyfy=True, chains=1, result_dir="../data/results"):
    all_results = []
    
    for algo_filename, algo_add in zip(algo_filenames, algo_additionals):
        algo_results = []
            
        for run in range(n_runs):
            run_str = "" if run == 0 else "_run{}".format(run)

            try:
                this_result = np.load(
                    "{}/{}_2_gan2d_{}{}_{}.npy".format(result_dir, algo_filename, algo_add, run_str, name)
                )
                if (not numpyfy) or (shape is None) or np.product(this_result.shape) == np.product(shape):
                    algo_results.append(this_result.reshape(shape))
                else:
                    algo_results.append(np.nan*np.ones(shape))

            except FileNotFoundError as e:
                # print(e)
                if shape is None:
                    algo_results.append(None)
                else:
                    algo_results.append(np.nan*np.ones(shape))
            
        all_results.append(algo_results)
    
    if numpyfy:
        all_results = np.array(all_results, dtype=np.float)
        all_results = all_results.reshape([all_results.shape[0], n_runs] + list(shape))
        
    return all_results


model_test_reco_xs = load("model_x_reco_test", (100, 3, 64, 64))
model_test_reco_errors = load("model_reco_error_test", (100,))
model_gen_fids = load("samples_fid", (1,)).squeeze()
model_gen_fids.shape


(4, 10)

In [5]:
max_reco_error = 10000.
model_mean_reco_errors = np.mean(np.clip(model_test_reco_errors, 0., max_reco_error), axis=2)
model_mean_reco_errors.shape


(4, 10)

## Outlier removal

In [6]:
def mean_err_without_outliers(data, remove=1):
    shape = list(data.shape)[:-1]
    data.reshape((-1, data.shape[-1]))
    
    means, errors = [], []
    
    for data_ in data:
        data_ = data_[np.isfinite(data_)]
        if not len(data_) > 0:
            means.append(np.nan)
            errors.append(np.nan)
            continue
            
        if len(data_) > 2*remove + 1:
            for _ in range(remove):
                data_ = np.delete(data_, np.argmin(data_))
                data_ = np.delete(data_, np.argmax(data_))

        means.append(np.mean(data_))
        errors.append(np.std(data_) / len(data_)**0.5)
        
    return np.array(means).reshape(shape), np.array(errors).reshape(shape)
    
    
model_fid_mean, model_fid_std = mean_err_without_outliers(model_gen_fids)
model_reco_error_mean, model_reco_error_std = mean_err_without_outliers(model_mean_reco_errors)


## Best metrics

In [7]:
best_fid = -1
best_reco = -1

best_fid = np.nanargmin(model_fid_mean)
print(algo_labels[best_fid])

best_reco = np.nanargmin(np.where(model_reco_error_mean > 1., model_reco_error_mean, np.nan))
print(algo_labels[best_reco])


\mfmfemd{}
\mfmfemd{}


## Print result table

In [8]:
def print_results(
    l_label=max([len(l) for l in algo_labels]), l_mean=5, l_err=4, latex=False, after_decs=(1,0), labels=["FID", "Reco error"]
):
    # Number of digits
    l_result = l_mean + 2 + l_err
    l_total = l_label + 1 + 2 * (3 + l_result)
        
    # Divider
    empty_result = "" if latex else " "*(l_result + 1)
    col_divider = "&" if latex else "|"
    line_end = r"\\" if latex else ""
    block_divider = r"\midrule" if latex else "-"*l_total
    
    # Number formatting
    def _f(val, err, after_dec, best=False):
        if not np.any(np.isfinite(val)):
            return empty_result
        
        result = "{:>{}.{}f}".format(val, l_mean, after_dec)
        if latex and best:
            result = r"\textbf{" + result + "}"
            
        err_str = "({:>{}.{}f})".format(err, l_err, after_dec)
        if latex:
            result += r"\,\textcolor{darkgray}{" + err_str + "}"
        else:
            result += err_str
            
        result += "*" if not latex and best else " "
        
        if latex:
            result = result.replace("-", "$-{}$")
            result = result.replace("darkgray", "dark-gray")
        return result
    
    
    # Header
    print(f"{'':<{l_label}.{l_label}s} {col_divider} {labels[0]:>{l_result}.{l_result}s} {col_divider} {labels[1]:>{l_result}.{l_result}s} {line_end}")
    print(block_divider)

    # Iterate over methods
    for i, (label, fid, fid_err, reco, reco_err) in enumerate(zip(
        algo_labels, model_fid_mean, model_fid_std, model_reco_error_mean, model_reco_error_std
    )):
        # Divider
        if i in algo_dividers:
            print(block_divider)
            
        # Print results
        print(
            f"{label:<{l_label}.{l_label}s} {col_divider} "
            + f"{_f(fid, fid_err, after_decs[0], i==best_fid):s}{col_divider} "
            + f"{_f(reco, reco_err, after_decs[1], i==best_reco):s} {line_end}"
        )


In [9]:
print_results()

           |         FID |  Reco error 
---------------------------------------
\af{}      |  55.8( 1.0) |     0(   0)  
\pie{}     | 143.3( 3.9) |  5631(  33)  
\mfmfmd{}  |  44.4( 0.0) |   346(   5)  
\mfmfemd{} |  43.4( 0.2)*|   304(   4)* 


In [10]:
print_results(latex=True)

           &         FID &  Reco error \\
\midrule
\af{}      &  55.8\,\textcolor{dark-gray}{( 1.0)} &     0\,\textcolor{dark-gray}{(   0)}  \\
\pie{}     & 143.3\,\textcolor{dark-gray}{( 3.9)} &  5631\,\textcolor{dark-gray}{(  33)}  \\
\mfmfmd{}  &  44.4\,\textcolor{dark-gray}{( 0.0)} &   346\,\textcolor{dark-gray}{(   5)}  \\
\mfmfemd{} & \textbf{ 43.4}\,\textcolor{dark-gray}{( 0.2)} & \textbf{  304}\,\textcolor{dark-gray}{(   4)}  \\


In [11]:
model_gen_fids

array([[ 58.20174533,  54.84207584,  54.31252978,  68.97241479,
         50.90466642,          nan,          nan,          nan,
                 nan,          nan],
       [152.73049773, 136.81524045, 140.40260621, 165.56874879,
        133.06243176,          nan,          nan,          nan,
                 nan,          nan],
       [ 44.33512519,  44.82221643,  43.70995254,  44.43353831,
         44.3712352 ,          nan,          nan,          nan,
                 nan,          nan],
       [ 43.45379455,  43.74331072,  42.8844404 ,          nan,
                 nan,          nan,          nan,          nan,
                 nan,          nan]])